# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Read in an Image

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images  
`cv2.cvtColor()` to grayscale or change color  
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [436]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def decipher_coords(img, line_coords):
    slope, intercept = line_coords
    y1 = int(img.shape[0])
    y2 = int(y1 * (3/5))
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)
    return (x1, y1, x2, y2)


def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    """
    Extrapolates lane lines
    
    Draws extrapolated lines on the masked region of interest
    """
    global p_l_x1, p_l_y1, p_l_x2, p_l_y2
    global p_r_x1, p_r_y1, p_r_x2, p_r_y2
    
    left_lane = []
    right_lane = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            coefficients = np.polyfit((x1, x2), (y1, y2), 1)
            slope, intercept = coefficients
            if slope > 0:
                right_lane.append(coefficients)
            else:
                left_lane.append(coefficients)
    
    # Find average slope and intercept of lines
    if left_lane:
        left_coefs = np.average(left_lane, axis=0)
        # Decipher x and y coordinates of the average of the coefficients
        l_x1, l_y1, l_x2, l_y2 = decipher_coords(img, left_coefs)
        # Record coordinates in immediate previous record
        p_l_x1, p_l_y1, p_l_x2, p_l_y2 = l_x1, l_y1, l_x2, l_y2
        # Draw extrapolated lines on the image
        cv2.line(img, (l_x1, l_y1), (l_x2, l_y2), color, thickness)
    else:
        cv2.line(img, (p_l_x1, p_l_y1), (p_l_x2, p_l_y2), color, thickness)
        
    if right_lane:
        right_coefs = np.average(right_lane, axis=0)
        # Decipher x and y coordinates of the average of the coefficients
        r_x1, r_y1, r_x2, r_y2 = decipher_coords(img, right_coefs)
        # Record coordinates in immediate previous record
        p_r_x1, p_r_y1, p_r_x2, p_r_y2 = r_x1, r_y1, r_x2, r_y2
        # Draw extrapolated lines on the image
        cv2.line(img, (r_x1, r_y1), (r_x2, r_y2), color, thickness)
    else:
        cv2.line(img, (p_r_x1, p_r_y1), (p_r_x2, p_r_y2), color, thickness)
    
    
# Define variables that preserve immediate previous lane line coords
p_l_x1, p_l_y1, p_l_x2, p_l_y2 = 0, 0, 0, 0
p_r_x1, p_r_y1, p_r_x2, p_r_y2 = 0, 0, 0, 0
 
    
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [455]:
import os
os.listdir("test_images/")

['solidYellowCurve.jpg',
 'solidYellowLeft.jpg',
 'solidYellowCurve2.jpg',
 'solidWhiteRight.jpg',
 'whiteCarLaneSwitch.jpg',
 'solidWhiteCurve.jpg']

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [461]:
# Find lane lines in image
def find_lane_lines(img):
    # Convert the image to gray scale
    gray = grayscale(img)
    # Blur the image
    blur = gaussian_blur(gray, 5)
    # Find image edges
    edges = canny(blur, 200, 250)
    # Find the region of interest from image
    img_height = edges.shape[0]
    vertices = np.array([
        [(50, img_height), (900, img_height), (500, 250)]
    ])
    roi_image = region_of_interest(edges, vertices)
    # Apply hough transform on the region of interest
    line_image = hough_lines(roi_image, 2, np.pi/180, 100, 40, 5)
    # Overlay the line image on the initial image
    weighted_image = weighted_img(line_image, img)
    
    
    # Return final image result
    return weighted_image

for img in os.listdir("test_images/"):
    image = mpimg.imread("test_images/" + img)
    lane_img = find_lane_lines(image)
    lane_img_name = "test_images_output/" + img
    cv2.imwrite(lane_img_name, lane_img)
    print("Saved", lane_img_name)

Saved test_images_output/solidYellowCurve.jpg
Saved test_images_output/solidYellowLeft.jpg
Saved test_images_output/solidYellowCurve2.jpg
Saved test_images_output/solidWhiteRight.jpg
Saved test_images_output/whiteCarLaneSwitch.jpg
Saved test_images_output/solidWhiteCurve.jpg


## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [447]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [448]:
def process_image(image):
    return find_lane_lines(image)

Let's try the one with the solid white lane on the right first ...

In [449]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)


















                

















             


















      



















                                                              

                                                              
                                                                 


                                                     



                                                        




                                                     





                                                  






                                               







                                            








                                         











                          














                  









                                      













                     










                              












                             















                    
















t:   2%|▏         | 5/2

Moviepy - Building video test_videos_output/solidWhiteRight.mp4.









                                               







                                            








                                         











                          














                  









                                      













                     










                              












                             















                    
















t:   2%|▏         | 5/221 [2:16:01<97:56:13, 1632.29s/it, now=None]

















t:   2%|▏         | 5/221 [2:15:01<97:13:16, 1620.35s/it, now=None]


















t:   2%|▏         | 5/221 [12:19<8:52:32, 147.93s/it, now=None]



















t:   2%|▏         | 5/221 [6:48:12<00:08, 24.21it/s, now=None]A

t:   2%|▏         | 5/221 [6:47:06<293:06:46, 4885.21s/it, now=None]


t:   2%|▏         | 5/221 [6:34:24<00:04, 49.05it/s, now=None]



t:   2%|▏         | 5/221 [6:30:58<281:30:23, 4691.77s/it, now=None]




t:   2%|▏         | 5/221

Moviepy - Writing video test_videos_output/solidWhiteRight.mp4
























t:   3%|▎         | 7/221 [00:00<00:03, 60.94it/s, now=None]




















t:   6%|▌         | 13/221 [00:00<00:03, 59.96it/s, now=None]




















t:   9%|▉         | 20/221 [00:00<00:03, 60.58it/s, now=None]




















t:  11%|█         | 24/221 [00:00<00:03, 52.02it/s, now=None]




















t:  13%|█▎        | 29/221 [00:00<00:03, 49.35it/s, now=None]




















t:  15%|█▌        | 34/221 [00:00<00:04, 46.34it/s, now=None]




















t:  18%|█▊        | 39/221 [00:00<00:03, 46.71it/s, now=None]




















t:  20%|██        | 45/221 [00:00<00:03, 48.97it/s, now=None]




















t:  23%|██▎       | 50/221 [00:01<00:05, 28.82it/s, now=None]




















t:  24%|██▍       | 54/221 [00:01<00:05, 29.94it/s, now=None]




















t:  26%|██▌       | 58/221 [00:01<00:05, 29.70it/s, now=None]




















t:  28%|██▊       | 62/221 [00:01<00:06, 23.04it/s, now=None]


















Moviepy - Done !
Moviepy - video ready test_videos_output/solidWhiteRight.mp4








t:   2%|▏         | 5/221 [6:18:48<272:44:23, 4545.66s/it, now=None]






t:   2%|▏         | 5/221 [6:16:33<271:07:20, 4518.70s/it, now=None]







t:   2%|▏         | 5/221 [6:11:24<267:24:39, 4456.85s/it, now=None]








t:   2%|▏         | 5/221 [5:28:11<236:17:55, 3938.31s/it, now=None]











t:   2%|▏         | 5/221 [5:27:11<00:04, 47.11it/s, now=None]














t:  69%|██████▉   | 86/125 [2:51:30<00:02, 14.34it/s, now=None]









t:   2%|▏         | 5/221 [5:27:44<235:58:45, 3932.99s/it, now=None]













t:  34%|███▍      | 86/251 [2:52:56<00:11, 14.76it/s, now=None]










t:   8%|▊         | 18/221 [5:27:26<00:04, 41.39it/s, now=None]












t:   2%|▏         | 5/221 [5:25:12<234:08:45, 3902.43s/it, now=None]















t:   2%|▏         | 5/221 [2:27:03<105:53:08, 1764.76s/it, now=None]

CPU times: user 3.15 s, sys: 603 ms, total: 3.75 s
Wall time: 9.78 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [452]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [453]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)


















                

















             


















      



















                                                              

                                                              
                                                                 


                                                     



                                                        




                                                     





                                                  






                                               







                                            








                                         











                          














                  









                                      













                     










                              












                             















                    
















t:   2%|▏         | 5/2

Moviepy - Building video test_videos_output/solidYellowLeft.mp4.
Moviepy - Writing video test_videos_output/solidYellowLeft.mp4










t:   2%|▏         | 5/221 [6:17:26<271:45:29, 4529.31s/it, now=None]







t:   2%|▏         | 5/221 [6:12:17<268:02:49, 4467.45s/it, now=None]








t:   2%|▏         | 5/221 [5:29:04<236:56:04, 3948.91s/it, now=None]











t:   2%|▏         | 5/221 [5:28:04<00:04, 47.11it/s, now=None]














t:  69%|██████▉   | 86/125 [2:52:23<00:02, 14.34it/s, now=None]









t:   2%|▏         | 5/221 [5:28:37<236:36:54, 3943.59s/it, now=None]













t:  34%|███▍      | 86/251 [2:53:49<00:11, 14.76it/s, now=None]










t:   8%|▊         | 18/221 [5:28:19<00:04, 41.39it/s, now=None]












t:   2%|▏         | 5/221 [5:26:05<234:46:55, 3913.03s/it, now=None]















t:   2%|▏         | 5/221 [2:27:56<106:31:17, 1775.36s/it, now=None]




















t:   0%|          | 0/681 [00:00<?, ?it/s, now=None]




















t:   1%|          | 8/681 [00:00<00:09, 71.00it/s, now=None]




















t:   2%|▏         | 15/681 [00:00<00:09, 69.16it/s,

Moviepy - Done !
Moviepy - video ready test_videos_output/solidYellowLeft.mp4
CPU times: user 8.41 s, sys: 1.51 s, total: 9.93 s
Wall time: 23.6 s


In [454]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))